
# How to generate text: using different decoding methods for language generation with Transformers

### **GPT-2**

#### ***Language Modeling***
- 기존 GPT-1은 pre-training과 supervised fine-tuning의 결합
- GPT-2는 language modeling(token sequence를 사용해 문장의 비지도 분포를 측정하는 방법)

#### ***Various Training Dataset***
- GPT-2의 가장 큰 목적은 Fine-tuning 없이 unsupervised pre-training 만을 Zero-shot으로 통해 다양한 task를 진행할 수 있는 General Language Model을 만드는 것
- GPT-1은 news article, wikipedia를 주로 사용. GPT-2는 Web Text 사용

#### ***Byte Pair Encoding(BPE)***
- subword 기반의 인코딩 방법으로 문자 단위로 단어를 분해해 vocab 생성
- OOV(Out Of Vocabulary) 문제 해결 

### **Introduction**

최근, 대량의 weppage 데이터로 학습 된 transformer-based language models의 등장으로 open-ended language generation에 대한 관심이 높아졌습니다.(ex. GPT-2) 

향상된 tranformer architecure와 대량의 unsupervised trainig data외에도 더 나은 decoding methods 또한 중요한 역할을 했습니다.


본 자료에서는 다양한 decoding strategies에 대한 간략한 개요와  `transformers` library를 사용해 손쉽게 이러한 decoding strategies를 구현하는 방법을 다룹니다.


모든 방법들은 **auto-regressive** language generation([here](http://jalammar.github.io/illustrated-gpt2/) a refresher)를 사용할 수 있습니다. 

![GPT-2 Auto regressive](http://jalammar.github.io/images/xlnet/gpt-2-autoregression-2.gif)

*Auto-regressive* language generation는 word sequence의 확률은 next word의 conditional distribution으로 decompose할 수 있다는 것이 기본 가정입니다. 
$$ P(w_{1:T} | W_0 ) = \prod_{t=1}^T P(w_{t} | w_{1: t-1}, W_0) \text{ ,with }  w_{1: 0} = \emptyset, $$

$W_0$는 initial *context* word sequence를 의미합니다. Word sequence의 길이인 $T$는 $P(w_{t} | w_{1: t-1}, W_{0})$으로 부터 EOS token이 나온 timestep $t=T$를 의미합니다.


Auto-regressive language generation은 `GPT2`, `XLNet`, `OpenAi-GPT`, `CTRL`, `TransfoXL`, `XLM`, `Bart`, `T5` in both PyTorch and Tensorflow >= 2.0!에서 사용 가능합니다.

이번 실습 시간에는 현재 유명한 decoding 방법들인 *Greedy search*, *Beam search*, *Top-K sampling*, *Top-p sampling* 를 다룹니다.

Transformer를 설치하고 Model을 load 해봅시다.

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.6 MB/s eta 0:00:00


이번 실습 시간에서는 SKT에서 공개한 KoGPT-2 모델을 사용합니다.

In [2]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2

Detected operating system as Ubuntu/focal.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.0.9
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages will be upgraded:
  git-lfs
1 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
Need to get 7,419 kB of archives.
After this operation, 4,936 kB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu focal/main amd64 git-lfs amd64 3.3.0 [7,419 kB]
Fetched 7,419 kB in 1s (7,858 kB/s)
(Reading database ... 122353 files and directories cu

In [3]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

tokenizer = SentencePieceBPETokenizer("/content/kogpt2/vocab.json", "/content/kogpt2/merges.txt")

# 이 모델같은 경우 vocab_size가 50000로 되어있음.
# 그래서 config GPT2 load를 할 때 반드시 vocab size를 맞춰줘야 함.
config = GPT2Config(vocab_size=50000)
config.pad_token_id = tokenizer.token_to_id('<pad>')
model = GPT2LMHeadModel(config)

model_dir = '/content/kogpt2/pytorch_model.bin'

# map_location을 해줘야 GPU에 올라가서 사용할 수 있다.
model.load_state_dict(torch.load(model_dir, map_location='cuda'), strict=False)
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50000, bias=False)
)

### **Greedy Search**

Greedy search는 다음 단어를 선택할 때, 가장 높은 확률을 가진 단어를 선택하는 단순한 방법입니다.

$w_t = argmax_{w}P(w | w_{1:t-1})$ at each timestep $t$. 

다음 그림은 greedy search를 나타낸 것 입니다.

![Greedy Search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/greedy_search.png)


알고리즘은 $\text{"The"}$에서 시작하여 가장 높은 확률을 가진 $\text{"nice"}$ 등을 선택하는 탐욕스러운(Greedy) 방법입니다. 따라서 최종적으로 생성 된 word sequence는 $\text{"The", "nice", "woman"}$이고 전체 확률은 $0.5 \times 0.4 = 0.2$ 입니다.

`Transformers` greedy search를 사용해봅시다.

In [4]:
# encode context the generation is conditioned on
def tokenizing(text):
    return torch.tensor(tokenizer.encode(text, add_special_tokens=False).ids).unsqueeze(0).to('cuda')

input_ids = tokenizing("포항은")

# generate text until the output length (which includes the context length) reaches 100
# 생성 모델은 generate 함수를 통해 다음 token을 생성해낼 수 있습니다.
# 그냥 넣어주면 자동으로 greedy search를 시작.
greedy_output = model.generate(input_ids, max_length=100)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output.tolist()[0], skip_special_tokens=True))


Output:
----------------------------------------------------------------------------------------------------
포항은 전반 16분 케빈의 선제골로 앞서갔지만 후반 16분 케빈에게 동점골을 허용했다.</s><s> 이 때문에 일각에서는 '안철수 효과'가 대선 이후 이어질지, 아니면 '박근혜 대세론'이 깨질지 등을 두고 '안풍'이 재연될 수 있다는 관측도 나오고 있다.</s><s> 안 후보는 이날 오전 공평동 캠프 사무실에서 열린 '새로운 정치, 새로운 변화를 위한 문재인의 구상'이라는 제목의 정책비전 발표회에서 "대통령이 되면 국민과 약속한 것을 반드시 실천하겠다"며


GPT2를 사용해 짧은 텍스트를 생성했습니다.

생성된 단어들의 문맥은 합리적이지만, 모델이 반복된 단어들을 생성합니다. 

이러한 현상은 일반적인 언어 생성 모델에서 나타나는 공통된 문제인데, 특히 Greedy Search와 Beam Search에서 그러한 현상이 더욱 두드러지게 나타납니다.

Greedy Search의 주요 단점은 그림에서 볼 수 있듯이 낮은 확률 단어 이후에 나올 수 있는 더 높은 확률의 단어를 놓친다는 점입니다. 

예를 들어 단어 $\text{has}$는 0.9로 높은 확률을 갖지만 첫번째 단어 후보 중 두번째로 높은 conditional probability를 갖는 $\text{dog}$ 이후에 숨어 있는 형태입니다. 따라서 Greedy Search는 $\text{"The"}, \text{"dog"}, \text{"has"}$ 라는 word sequence를 놓치게 됩니다.

### **Beam search**

Beam search는 각 time step에서 가장 확률이 높은 hypothesis의 `num_beams`를 유지하고 전체 확률이 가장 높은 hypothesis를 선택하는 방법입니다. 즉, 해당 시점에서 유망한 빔의 개수만큼(num_beams) 골라서 진행하는 방식으로 높은 확률을 가지고 있지만 숨어있는 word sequence를 놓칠 위험을 줄입니다.다음 그림은 `num_beams=2`로 설정한 Beam search의 예시입니다:

![Beam search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/beam_search.png)


Time step $1$ : 가장 가능성이 높은 hypothesis인 $\text{"The", "nice"}$ 외에도 beam search는 두번째로 가능성이 높은 $\text{"The", "dog"}$를 추적합니다. 

Time step $2$ : beam search는 $0.2$의 가능성을 가진  $\text{"The", "nice", "woman"}$보다 $0.36$의 가능성을 가진 $\text{"The", "dog", "has"}$가 확률이 더 높다는 것을 찾습니다. 

이 방법은 우리의 toy example에서 가장 가능성이 높은 word sequence를 찾아냈습니다.

Beam search는 항상 Greedy search보다 높은 확률의 결과 sequence를 찾는 것이 가능하지만, 이게 가장 가능성이 높은 결과라고는 보장할 수 없습니다.

`transformers`에서 beam search를 사용하는 방법을 살펴 봅시다. 우리는 `num_beams > 1`, `early_stopping=True` 으로 설정하여 모든 beam hypothesis가 eos토큰에 닿으면 생성을 마치도록 합니다.

In [5]:
# activate beam search and early_stopping
beam_output = model.generate(
    input_ids,  
    max_length=100, 
    num_beams=5, 
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output.tolist()[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
포항은 2일 오후 7시 포항 스틸야드에서 포항 스틸러스와 K리그 클래식 25라운드 원정경기를 갖는다.</s><s> 한편, 이날 방송에서는 이숙영(이경숙 분)과 강지녕(이보영 분)의 러브스토리가 전파를 탔다.</s><s> [사진]류현진,'다저스 타선이 도와줘'</s><s> 다저스는 2일(한국시간) 미국 로스앤젤레스 다저스타디움에서 열리는 신시내티 레즈와의 경기에 류현진을 선발


결과는 더 유창하게 보이지만 여전히 동일한 word sequence를 반복하는 문제를 포함합니다. 

단순한 해결법은 [Paulus et al. (2017)](https://arxiv.org/abs/1705.04304)와 [Klein et al. (2017)](https://arxiv.org/abs/1701.02810)의 논문에서 제안 된 n-grams 페널티를 도입하는 것입니다. 가장 일반적인 n-grams 페널티는 이미 나타난 n-gram에 대해 다음 단어로 생성 될 확률을 0으로 설정하여 두번 나타나지 않도록 하는 방법입니다. 

`no_repeat_ngram_size=2`를 설정해 2-gram이 두번 나타나는 것을 막을 수 있습니다.

In [6]:
# set no_repeat_ngram_size to 2
beam_output = model.generate(
    input_ids, 
    max_length=100, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output.tolist()[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
포항은 지난 2일 포항 스틸야드에서 열린 FA컵 32강 포항스틸러스와 경기서 1-1 무승부를 거뒀다.</s><s> 이 같은 사실은 지난 3일 방송된 MBC ‘황금어장-무릎팍도사’에서 한혜진이 한혜진과의 열애 사실을 공개한 후 급속도로 퍼지게 됐다”고 전했다. tvN ‘화성인 X파일’에 출연한 화성인은 “한혜진과 축구선수 기성용 선수가 열애설이 있다”며 “기성용과 열애설은 사실무근이다”


더이상 반복이 나타나지 않는 것을 볼 수 있습니다.

하지만, n-gram 페널티는 신중하게 사용되어야 합니다. 예를 들어, city New York에 대해 생성된 기사는 n-gram을 사용하지 않는 것이 좋습니다. 2-gram을 사용하게 될 경우 시의 이름이 전체 텍스트에서 한 번만 나타나기 때문입니다.

Beam search의 또 다른 중요한 특징은 생성된 Top beam을 비교하여 목적에 가장 적합한 Beam을 선택할 수 있다는 것입니다.

`Transformers`에서 우리는 `num_return_sequences`를 top-n개의 높은 scoring을 가진 beam을 return할 것인지 설정 할 수 있습니다.

단, num_return_sequences는 num_beams보다 같거나 작아야합니다.

In [7]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, 
    early_stopping=True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output.tolist(), skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: 포항은 지난 2일 포항스틸야드에서 열린 포항 스틸러스와의 2012년 K-리그 35라운드에서 1대1로 비겼다.</s><s> 한편, 이날 방송에서는 오영실(송옥숙 분)이 오수(조인성
1: 포항은 지난 2일 포항스틸야드에서 열린 포항 스틸러스와의 2012년 K-리그 35라운드에서 1대1로 비겼다.</s><s> 한편, 이날 방송에서는 오영실(송옥숙 분)과 오수성(오
2: 포항은 지난 2일 포항스틸야드에서 열린 포항 스틸러스와의 2012년 K-리그 35라운드에서 1대1로 비겼다.</s><s> 한편, 이날 방송에서는 오영실(송옥숙 분)이 오수성(오
3: 포항은 지난 2일 포항스틸야드에서 열린 포항 스틸러스와의 2012년 K-리그 35라운드에서 1대1로 비겼다.</s><s> 한편, 이날 방송에서는 오영실(송옥숙 분)이 오수성(성
4: 포항은 지난 2일 포항스틸야드에서 열린 포항 스틸러스와의 2012년 K-리그 35라운드에서 1대1로 비겼다.</s><s> 한편, 이날 방송에서는 오영실(송옥숙 분)이 영희(영


위에서 볼 수 있듯이 Top 5의 Beam hypothesis는 서로 약간만 다를 뿐이며 5개만 사용했을 경우 별로 놀랄만한 결과는 아닙니다.

Open-ended generation에서, beam search가 최선의 선택이 아닐 수 있는 몇가지 이유가 제시 되었습니다.

- Beam search는 Machine translation 또는 Text summarization처럼 원하는 문장 생성 길이가 예측 가능한 task에서는 잘 작동 될 수 있습니다. 하지만 Dialog 또는 Story generation task처럼 출력 길이가 크게 달라질 수 있는 open-ended generation에서는 원활하게 작동하지 않습니다 - see [Murray et al. (2018)](https://arxiv.org/abs/1808.10006) and [Yang et al. (2018)](https://arxiv.org/abs/1808.09582). 

- Beam search는 반복 생성 문제에 취약합니다. 특히 Story generation task에서 n-gram 또는 기타 페널티를 통해 문장을 제어하는 것이 어렵습니다. *반복이 없는 구문* 과 *n-gram*의 반복 주기 사이에서 적당한 trade-off를 찾기 위해 많은 fine-tuning이 필요하기 때문입니다.

- [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751)는 고품질 인간 언어는 높은 확률의 다음 단어 분포를 따르지 않는다고 주장합니다. 쉽게 말하자면 인간 입장에서 우리는 지루하거나 예측 가능한 문장이 아니라 우리를 놀라게 할 수 있는 문장 생성을 원한다고 합니다. 저자는 모델이 human text vs. beam seach를 graph로 보여주면서 beam search text가 그다지 놀랍지 않은 문장임을 보여주었습니다.

![alt text](https://blog.fastforwardlabs.com/images/2019/05/Screen_Shot_2019_05_08_at_3_06_36_PM-1557342561886.png)


So let's stop being boring and introduce some randomness 🤪.

### **Sampling**

가장 기본적인 형태의 sampling은 조건부 확률 분포에 따라 다음 단어 $w_t$를 무작위로 선택하는 것을 의미합니다.

$$w_t \sim P(w|w_{1:t-1})$$

아래 사진은 sampling 할 때, 언어 생성을 시각화한 형태입니다.

![vanilla_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/sampling_search.png)

Sampling을 이용한 언어 생성은 더이상 *deterministic*하지 않습니다. 

단어 $\text{"car"}$ 는 contional probability distribution $P(w | \text{"The"})$에서 샘플링 되고, $P(w | \text{"The"}, \text{"car"})$는 $\text{"drives"}$를 샘플링 합니다.


`Transformers`에서 우리는 `do_sample=True` 를 설정하고 `top_k=0`로 두어 *Top-K* 를 비활성화합니다.(뒤에서 다룰 것)



In [8]:
# torch.manual_seed(53) #원한다면 random seed를 지정 할 수 있습니다.

# activate sampling and deactivate top_k by setting top_k sampling to 0

sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output.tolist()[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
포항은 반면, 대안동은 설욕 의지를 앞세워 상승세다.</s><s> 아이린이 생후 7개월 된 나에게는 지금이라도 출간된 아동도서선물을 곁들이는 재능기부로 보답했다.</s><s> 고소진은 또 힐링캠프에서 매일 같이 라면서 그림책 100


괜찮아 보이지만 일관성이 없습니다. 이것은 sampling word sequences를 할때 모델이 일관성없이 횡설수설하는 문장을 발생시키는 큰 문제입니다. ([Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751)).
모델이 만들어 낸 확률이 smooth한 나머지, 낮은 확률의 토큰이 지나치게 잘 샘플링 되는 것이 원인 중 하나입니다.


한가지 트릭은 [softmax](https://en.wikipedia.org/wiki/Softmax_function#Smooth_arg_max) 의 이른바 `temperature`를 낮추어 분포 $P(w|w_{1:t-1})$를 더 선명하게 만드는 것입니다. 높은 확률의 단어의 가능성은 증가시키고 낮은 확률의 단어 가능성은 감소시키는 효과가 있습니다. 
temperature가 0에 가까워질 수록 greedy decoding에 가까운 아웃풋이 나옵니다.

temperature를 적용한다면 다음과 같은 그림을 보일 수 있습니다.


![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/sampling_search_with_temp.png?raw=true)

step=1의 다음 단어 분포는 더욱 선명해졌기 때문에 단어 $\text{"car"}$를 선택할 확률이 거의 없습니다.


`temperature=0.7`를 설정하여 라이브러리에서 분포를 어떻게 변화시키는지 알아보겠습니다.

In [9]:
# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=100, 
    top_k=0, 
    temperature=0.7
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output.tolist()[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
포항은 대구FC와의 홈경기에서 1-1 무승부를 거뒀다.</s><s> 또한, 대구ONE FC는 오는 26일 인천 유나이티드로 FC 서울과의 홈경기를 앞두고 있다.</s><s> 경기 전 한혜진은 "약속한 대로 샵이든, 카페든 안 가면 되지 않고 그냥 만나자"며 메세지를 전했다.</s><s> 한편, 한혜진은 오는 7월 1일 기성용과 서울 강남구 삼성동 인터콘티넨탈 호텔에서 혼전임신과 축구선수 기성용과의


이제 이상한 n-gram이 적고 출력 문장이 조금 더 일관성 있게 생성됩니다. temperature를 적용하면 분포가 덜 random하지만 `temperature` $ \to 0$로 설정한다면 temperature가 적용된 sampling은 greedy decoding과 같아지며 이전과 동일한 문제를 겪게 됩니다.

### **Top-K Sampling**

[Fan et. al (2018)](https://arxiv.org/pdf/1805.04833.pdf) 은 간단하지만 매우 강력한 샘플링 방식을 도입했습니다. . *Top-K* sampling에서 높은 가능성을 가진 k개를 제외한 단어는 필터링 되고 k 이후의 probablity mass는 재분배됩니다. GPT2는 Top-K Sampling방식을 채택했는데, 이것이 Story Gerneration Task에 성공한 이유중 하나입니다.

Top-K Sampling을 더 잘 설명하기 위해 위의 예제에서 두 Sampling step에 사용되는 범위를 3단어에서 10단어로 확장합니다.

![top_k_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/top_k_sampling.png)


K=6을 설정하면 두 Sampling steps에서 Sampling pool을 6개의 단어로 제한합니다.

$\text{"The"}$ 다음으로 나올 수 있는 6개를 선택하고 $\text{"The:, "car"}$ 뒤에 올 수 있는 6개를 선택합니다. 

첫 step에서 전체 확률 질량의 2/3인 0.68정도에 해당하는 단어에서 디코딩되지만, 두번째 step에서 거의 모든 확률질량인 0.99에서 디코딩합니다.

그럼에도 불구하고 그것이 두번째 sampling step에서 $\text{"not", "the", "small", "told"}$ 와 같은 다소 이상한 후보들을 성공적으로 제거가 가능했습니다.


In [10]:
# set top_k to 50
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output.tolist()[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
포항은 울산과 0-0 무승부, 대구는 전북의 홈 앤드 어웨이 4강에 진출했다.</s><s> 이어 "이들이 주장하는 것이 바로 바로 한 번도 경험하지 못한 새로운 방식으로 우리의 생각을 표현하고 우리 국민에게 카타르시스를 드리며 새로운 미래를 만드는


지금까지 봐온 decoding methods 중 가장 사람다워 보이는 텍스트를 생성했습니다. Top-K Sampling의 한가지 우려되는 점은 다음 단어 확률 분포 $P(w|w_{1:t-1})$에서 필터링 된 단어 수를 동적으로 조정하지 않는 점입니다. 예를들어 위 그림에서 첫번째 step의 단어들은 전반적으로 평평한 분포에서 sampling되지만, 두번째 step의 어떤 단어들은 매우 sharp한 분포에서 sampling 될 수 있기 때문에 문제가 될 수 있습니다.

Step $t=1$에서 Top-K은 꽤 합리적인 후보처럼 보이는 $\text{"people", "big", "house", "cat"}$을 샘플링하는 가능성을 배제합니다. 반면에 Step $t=2$에서 단어 Sample pool에 단어 $\text{"down", "a"}$와 같은 부적절한 단어를 포함합니다. 그러므로 Sample pool이 고정크기 K로 제한되면 모형이 Sharp한 분포에서 횡설수설한 단어를 고를 위험이있고 평평한 분포에서는 문장의 창의성이 제한될 수 있습니다. ([Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751))


### **Top-p (nucleus) sampling**

*Top-p* sampling은 가장 가능성이 높은 K개에서만 sample을 추출하는 것이 아니라 누적 확률이 확률 p를 초과하는 최소한의 단어 집합에서 sample을 추출합니다.

그 후 확률 질량이 단어 집합 사이에 재분배 됩니다. 이 방법은 다음 단어의 확률 분포에 따라 단어 집합의 크기가 동적으로 증가하거나 감소할 수 있습니다.


![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/top_p_sampling.png?raw=true)

$p=0.92$로 설정할 경우, *Top-p* 는 $p=92\%$를 초과할 수 있는 최소 단어 수를 선택합니다. 첫번째 스텝에서 가장 가능성 높은 단어 9개가 포함된 반면, 두번째 스텝에서는 top 3개만 선택해도 $p=92\%$를 초과하게 됩니다. 즉, 높은 확률의 단어에만 sampling을 하고 그렇지 않은 단어는 sampling할 확률이 매우 적어집니다.

`Transformers`에서 `top_p ∈ (0,1)`을 설정하여 *Top-p* sampling을 설정할 수 있습니다.

In [11]:
# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_p=0.92, 
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output.tolist()[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
포항은 골을 넣으면 4만 달러의 상금을 거니 인수권을 따낸다는 원칙을 세웠다.</s><s> 이와 관련해 전문가들은 지난 2005년 개정된 국내항공모함법의 위헌성 여부 판결 당시 헌재의 입장은 합헌일 가능성이 크다고 보고 있다.</s><s> 어느


이론적으로는 *Top-p*가 *Top-K*보다 성능이 좋아 보이지만, 두 방법 모두 실제로 잘 작동합니다. 

*Top-p*와 *Top-K*는 함께 사용될 수 있는데, 이는 매우 낮은 순위의 단어를 피하면서도 일부 동적 선택을 허용할 수 있습니다.

독립적으로 sampling된 multiple outputs를 얻기 위해 `num_return_sequences > 1`로 설정해봅시다.

In [12]:
# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True, 
    max_length=50, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output.tolist(), skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: 포항은 25일 홈 구장인 포항 스틸러스서 열린 포항 스틸러스와 4강 2차전 홈경기에 케빈과 김형준을 영입하며 공격진의 파괴력을 예고했다.</s><s> 또 같은 날 열린 1차전에서는 K리그 클래식 3연
1: 포항은 최근 K리그 클래식 2경기에서 2승 2무를 거뒀다.</s><s> "경찰 수사자료, 녹취록 분석결과 발표하겠다"</s><s> "경찰 수사자료, 녹취록 분석결과 발표하겠다"</s><s> '내란음모' 혐의로 수사를 받고
2: 포항은 2012시즌부터 ‘현대오일뱅크 K리그’의 스폰서를 맡으며 울산은 더욱 강해진 모습을 보여주고 있다.</s><s> 하지만 '신사의 품격'의 윤세아, 김지석, 김재원, 이종혁, 윤상현 등이 '다
3: 포항은 3일 오후 6시 터키 앙카라 구장에서 알 이티무와 친선경기를 치를 예정이었지만 울산으로 연고지가 변경됐다.</s><s> 그러나 이 영화는 애초 2008년 7월에 개봉될 예정이었으나, 그전에 다른 배우들이 캐스팅 문제로 갈등을 빚
4: 포항은 이겼지만 2경기에서는 골득실에서 밀려 2위를 달리는 등 최근 5년 동안 정규리그에서 단 한 번도 4강에 들지 못하는 부진을 겪고 있다.</s><s> ‘내 딸 서영이’의 이상윤 역시 자신의 마음을 숨길 수


Cool, now you should have all the tools to let your model write your stories with `transformers`!

### **Conclusion**

*ad-hoc* decoding methods에 따르면 open-ended generation에서 *top-p* and *top-K*는 *greedy*, *beam search*보다 더욱 유창한 text를 생성합니다.

하지만  [Welleck et al. (2020)](https://arxiv.org/abs/2002.02492)에 따르면 *top-K*와 *top-p*는 여전히 반복되는 word sequences를 생성하는 문제(*greedy*, *beam search*와 같이)를 겪고 있다고 합니다.

[Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf)는 human evalutation 관점에서, model training 목적 함수를 잘 조정하면, *beam search*가 *Top-p*보다 유창한 text를 생성한다고 주장합니다.

Open-ended language generation은 빠르게 발전하는 분야이며, 무엇이 적합하다고 단정할 수 없으므로 특정 사용 사례에서 가장 잘 작동하는 방법이 무엇인지 고려해야합니다.

### **Appendix**

There are a couple of additional parameters for the `generate` method that were not mentioned above. We will explain them here briefly!

- `min_length` can be used to force the model to not produce an EOS token (= not finish the sentence) before `min_length` is reached. This is used quite frequently in summarization, but can be useful in general if the user wants to have longer outputs.
- `repetition_penalty` can be used to penalize words that were already generated or belong to the context. It was first introduced by [Kesker et al. (2019)](https://arxiv.org/abs/1909.05858) and is also used in the training objective in [Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf). It can be quite effective at preventing repetitions, but seems to be very sensitive to different models and use cases, *e.g.* see this [discussion](https://github.com/huggingface/transformers/pull/2303) on Github.

- `attention_mask` can be used to mask padded tokens
- `pad_token_id`, `bos_token_id`, `eos_token_id`: If the model does not have those tokens by default, the user can manually choose other token ids to represent them.

For more information please also look into the `generate` function [docstring](https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.TFPreTrainedModel.generate).